In [141]:
from heapq import heappop, heappush, heapify

def calc_manhattan_distance(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    return(abs(x1-x2)+abs(y1-y2))

class Maze():
    def __init__(self, maze_grid):
        self.maze_grid = maze_grid
        self.maze_list = self.maze_unpack(maze_grid)
        self.start = [0,0]
        self.length = len(self.maze_list)
        self.finish = [self.length-1,self.length-1]
        self.start_value = int(self.maze_list[0][0])
        
    def get_adjacent_cells(self, Node):
        
        maze = self.maze_list
        maze_l = self.length
        
        x,y = Node.coord
        cng = Node.value
        
        def cngdf(val):
            return(abs(int(val) - int(cng)))
        
        adj_cells = [] #list of lists [[new x, new y, cost of move, value from maze],[...],...]
        #check right
        if x + 1 < maze_l : adj_cells.append([x+1, y, cngdf(maze[y][x+1]),maze[y][x+1]])
        #check left
        if x - 1 >= 0 : adj_cells.append([x-1, y, cngdf(maze[y][x-1]),maze[y][x-1]])
        #check down
        if y + 1 < maze_l : adj_cells.append([x, y + 1, cngdf(maze[y + 1][x]),maze[y + 1][x]])
        #check up
        if y - 1 >= 0 : adj_cells.append([x, y-1, cngdf(maze[y-1][x]),maze[y-1][x]])
        return(adj_cells)
    
    #take maze from codewars format into list of lists
    def maze_unpack(self, maze_grid):
        return(maze_grid)
        #return([list(i) for i in  self.maze_grid.split('\n')])


class Node():
    def __init__(self, coord, maze, g, value):
        self.x = coord[0]
        self.y = coord[1]
        self.g = g
        self.value = value
        self.h = calc_manhattan_distance(coord, maze.finish)
        self.coord = coord
        self.parent = None
        self.coord_t = tuple(coord)
        
    #compute total costs from start plus estimated costs to finish
    def f(self):
        return(self.g + self.h)
    
def retrace_path(node):
    total = node.g
    while node.coord !=[0,0]:
        print(node.coord)
        print(total - node.g)
        print(node.g)
        print(node.value)
        print("******")
        node = node.parent
    
def path_finder(maze):
    closed = set() #closed coords
    frontier = [] #heap of costs and node coordinates
    node_finder = {} #dict of coords and node objs #node_finder[coord] = Node obj
    
    maze = Maze(maze)
    #print(maze.maze_list)
    start_node = Node([0,0], maze, 0, maze.start_value)
    
    #push a list of [costs, [x,y]]
    heappush(frontier, [start_node.f(), start_node.coord])
    
    #add coords to lookup dict to ref node object
    node_finder[start_node.coord_t] = start_node
    
    #while we have nodes...
    while frontier:
        
        #pop the cheapest node off for testing
        current_node = heappop(frontier)
        current_node = node_finder[tuple(current_node[1])]
        
        print("current_node coord = %s" % current_node.coord)
        print("current_node value = %s" % current_node.value)
        print("current_node g = %s" % current_node.g)
        print("current_node h = %s" % current_node.h)
        print("current_node f = %s" % current_node.f())
        print("\n")
              
        #if at finish coords
        if current_node.coord == maze.finish:
            print(retrace_path(current_node))
            print(current_node.g)
            return(current_node.g)
        
        #add the coords to our closed/visited/seen set
        closed.add(current_node.coord_t)
        
        #make a list of valid neighbors to visit, only major cardinal directions with no walls
        adj_nodes = [Node(i[0:2], maze, i[2], i[3]) for i in maze.get_adjacent_cells(current_node)]
        
        for node in adj_nodes:   
            
            #if already tested node, skip
            if tuple(node.coord_t) in closed:
                continue            
            
            #else evaluate
            else:
                print("coord = %s" % node.coord)
                print("value = %s" % node.value)
                print("g = %s" % node.g)
                print("h = %s" % node.h)
                print("f = %s" % node.f())
            
                #add cost of move to this neighbor node to the cost of the total path here
                node.g = current_node.g + node.g
                print("new node g = %s" % node.g)
                print("\n")
                
                #record parent node
                node.parent = current_node
                
                #check the dict to see if this total cost is less than the existing path cost to this node
                #if so, replace the node details and add the new details to the frontier
                if node.coord_t in node_finder:
                    old_node_g = node_finder[node.coord_t].g
                    old_node_f = node_finder[node.coord_t].f()
                    if node.g < old_node_g:
                        node_finder[node.coord_t] = node
                        index_node_match = frontier.index([old_node_f, node.coord]) # get the index of node on frontier so f can be replaced
                        print("*Faster Path* = %s" % frontier.index([old_node_f, node.coord]))
                        frontier[index_node_match] = [node.f(), node.coord] # replace the f value with shorter one and same node coords
                        heapify(frontier)
                        
                        #could also just push onto heap again
                        #heappush(frontier, [node.f(), node.coord])
                        
                #otherwise, add neighbor nodes to the dict and to the frontier for further exploration
                else:
                    node_finder[node.coord_t] = node
                    heappush(frontier, [node.f(), node.coord])
    return(False)

#should = 72, equals 74 instead.  passes 149 of 151 randomized test cases
a=[['6', '5', '9', '2', '3', '5', '5', '1', '0', '3', '8', '2', '1', '0', '6', '4', '1', '7', '1', '4', '5', '7', '1', '3', '1', '2', '2', '0'], 
   ['1', '6', '0', '7', '5', '1', '0', '1', '8', '4', '9', '6', '8', '5', '2', '9', '7', '1', '2', '4', '3', '3', '1', '9', '1', '1', '6', '9'], 
   ['0', '6', '5', '2', '3', '9', '3', '5', '1', '9', '0', '3', '4', '3', '8', '8', '6', '6', '5', '0', '6', '0', '8', '3', '5', '8', '2', '0'], 
   ['0', '4', '8', '4', '4', '7', '6', '3', '2', '1', '2', '9', '1', '5', '2', '7', '0', '9', '4', '9', '1', '9', '4', '7', '3', '5', '0', '7'], 
   ['9', '4', '1', '8', '2', '7', '3', '6', '2', '8', '0', '1', '5', '5', '4', '4', '5', '8', '9', '6', '3', '6', '2', '3', '5', '2', '1', '9'], 
   ['0', '1', '7', '6', '2', '5', '9', '6', '9', '4', '1', '0', '3', '9', '8', '7', '5', '9', '1', '5', '1', '6', '3', '6', '5', '5', '9', '7'],
   ['4', '2', '3', '9', '2', '8', '5', '9', '0', '2', '7', '3', '1', '9', '5', '1', '8', '3', '1', '7', '7', '7', '5', '7', '5', '9', '7', '8'],
   ['5', '1', '9', '7', '1', '8', '0', '1', '0', '3', '1', '1', '9', '2', '2', '6', '5', '2', '9', '0', '5', '6', '8', '6', '7', '0', '1', '9'], 
   ['2', '2', '1', '0', '1', '4', '8', '8', '5', '1', '7', '2', '2', '9', '8', '2', '3', '3', '1', '3', '2', '3', '1', '0', '6', '1', '4', '5'], 
   ['5', '9', '5', '6', '5', '1', '9', '3', '3', '3', '4', '5', '4', '1', '5', '5', '4', '1', '0', '4', '1', '9', '7', '4', '4', '1', '6', '8'], 
   ['6', '1', '3', '1', '3', '9', '0', '9', '2', '2', '7', '7', '9', '9', '0', '4', '6', '7', '7', '5', '8', '4', '3', '1', '2', '3', '7', '1'],
   ['0', '8', '0', '1', '4', '2', '1', '3', '0', '4', '9', '9', '0', '7', '8', '9', '7', '7', '2', '2', '7', '7', '6', '5', '9', '5', '2', '5'], 
   ['6', '4', '0', '4', '4', '8', '6', '8', '1', '2', '2', '8', '3', '7', '9', '7', '4', '6', '5', '0', '0', '9', '6', '6', '1', '3', '1', '8'], 
   ['4', '7', '2', '7', '5', '3', '0', '0', '2', '1', '4', '6', '2', '2', '6', '5', '9', '8', '7', '8', '6', '2', '2', '9', '2', '6', '6', '8'], 
   ['6', '0', '6', '0', '7', '4', '2', '2', '0', '9', '6', '3', '8', '5', '2', '8', '3', '1', '5', '4', '3', '8', '2', '1', '6', '1', '3', '0'], 
   ['2', '9', '2', '1', '0', '5', '5', '1', '7', '2', '7', '0', '6', '2', '9', '8', '2', '5', '6', '2', '4', '6', '9', '5', '7', '8', '9', '4'], 
   ['0', '1', '2', '7', '5', '4', '2', '0', '8', '5', '9', '0', '8', '3', '0', '9', '1', '4', '1', '7', '6', '2', '9', '3', '8', '2', '8', '6'], 
   ['4', '8', '1', '8', '4', '1', '5', '7', '9', '9', '1', '5', '5', '5', '4', '0', '0', '4', '8', '5', '1', '0', '1', '7', '8', '1', '6', '5'], 
   ['7', '5', '7', '8', '9', '5', '9', '3', '7', '0', '6', '1', '0', '1', '7', '0', '9', '4', '9', '8', '3', '3', '0', '0', '7', '4', '3', '3'], 
   ['5', '6', '7', '0', '3', '6', '5', '4', '6', '2', '6', '2', '1', '0', '9', '9', '5', '5', '7', '2', '9', '7', '5', '7', '4', '0', '2', '7'], 
   ['0', '8', '8', '4', '4', '1', '6', '1', '6', '0', '7', '3', '3', '4', '6', '3', '7', '8', '5', '6', '3', '6', '8', '3', '5', '5', '7', '2'], 
   ['3', '2', '6', '6', '6', '9', '4', '2', '4', '0', '9', '9', '6', '0', '8', '7', '9', '7', '1', '4', '9', '2', '8', '8', '1', '9', '5', '1'], 
   ['2', '5', '5', '9', '6', '5', '4', '6', '8', '6', '5', '0', '0', '4', '6', '8', '8', '0', '9', '5', '5', '2', '2', '3', '3', '3', '3', '0'], 
   ['3', '2', '9', '4', '0', '2', '7', '9', '8', '0', '5', '0', '8', '0', '5', '5', '3', '9', '6', '3', '5', '5', '4', '8', '5', '8', '9', '1'], 
   ['5', '8', '6', '4', '0', '2', '7', '3', '7', '1', '1', '5', '3', '5', '2', '5', '5', '1', '6', '6', '3', '8', '9', '7', '9', '9', '7', '1'],
   ['9', '9', '2', '6', '0', '9', '8', '8', '2', '2', '6', '3', '9', '3', '3', '7', '3', '7', '8', '8', '8', '6', '8', '8', '2', '5', '4', '9'],
   ['2', '1', '4', '0', '2', '6', '5', '0', '9', '9', '7', '4', '2', '8', '5', '8', '8', '5', '6', '3', '2', '2', '0', '2', '5', '5', '8', '9'], 
   ['5', '4', '2', '6', '2', '8', '3', '9', '8', '0', '5', '3', '7', '7', '6', '5', '2', '0', '4', '0', '9', '1', '9', '5', '7', '1', '3', '4']]

In [142]:
#59 should be 57
a = [['2', '8', '1', '9', '9', '5', '0', '4', '9', '0', '4', '9', '4', '7', '5', '2', '2', '8', '8', '9', '1'], 
     ['2', '4', '6', '0', '9', '5', '4', '2', '3', '4', '4', '8', '8', '0', '4', '3', '1', '4', '2', '4', '8'], 
     ['4', '8', '1', '3', '3', '2', '0', '4', '3', '3', '9', '3', '7', '4', '1', '7', '7', '8', '7', '9', '9'], 
     ['8', '6', '9', '3', '8', '3', '8', '8', '1', '7', '4', '1', '2', '7', '2', '0', '9', '9', '0', '7', '3'], 
     ['3', '5', '1', '7', '2', '2', '4', '8', '8', '2', '0', '2', '0', '4', '0', '0', '5', '7', '9', '3', '9'], 
     ['4', '1', '2', '7', '6', '4', '9', '3', '7', '4', '0', '5', '0', '6', '3', '9', '6', '6', '4', '2', '3'], 
     ['8', '7', '6', '5', '3', '5', '0', '0', '9', '3', '8', '8', '3', '3', '7', '3', '2', '7', '9', '5', '5'], 
     ['2', '6', '1', '4', '2', '4', '6', '9', '3', '9', '3', '1', '6', '1', '9', '7', '2', '8', '7', '8', '7'], 
     ['6', '9', '6', '4', '4', '4', '2', '8', '2', '0', '6', '5', '2', '2', '3', '2', '9', '2', '9', '1', '6'], 
     ['0', '9', '9', '9', '9', '0', '0', '3', '2', '7', '2', '3', '2', '0', '5', '6', '1', '6', '1', '1', '8'], 
     ['6', '5', '2', '8', '8', '4', '7', '7', '9', '4', '0', '7', '3', '4', '1', '6', '2', '6', '0', '1', '8'], 
     ['7', '7', '2', '7', '7', '6', '3', '0', '0', '1', '0', '5', '1', '3', '9', '1', '3', '1', '9', '2', '8'], 
     ['5', '8', '3', '1', '4', '8', '0', '9', '5', '9', '9', '8', '6', '5', '4', '0', '6', '2', '9', '4', '9'], 
     ['9', '0', '3', '7', '8', '9', '1', '1', '0', '1', '1', '8', '0', '3', '0', '6', '7', '3', '9', '5', '1'], 
     ['8', '7', '8', '1', '1', '7', '4', '4', '6', '1', '0', '6', '7', '6', '1', '1', '4', '0', '9', '6', '9'], 
     ['0', '3', '3', '2', '5', '9', '0', '3', '1', '9', '8', '3', '5', '1', '8', '4', '3', '2', '4', '0', '4'], 
     ['1', '7', '9', '5', '2', '3', '7', '9', '6', '0', '0', '7', '9', '0', '1', '8', '7', '7', '8', '8', '0'], 
     ['3', '8', '1', '4', '2', '7', '9', '7', '3', '5', '1', '3', '9', '1', '6', '3', '1', '0', '6', '8', '1'],
     ['3', '5', '0', '6', '2', '9', '7', '3', '0', '9', '7', '4', '9', '8', '7', '1', '1', '5', '3', '1', '3'], 
     ['6', '9', '5', '9', '3', '8', '8', '9', '9', '6', '9', '2', '2', '9', '4', '5', '6', '5', '3', '7', '4'], 
     ['2', '1', '1', '1', '3', '3', '7', '0', '8', '1', '5', '5', '0', '9', '4', '0', '6', '5', '6', '6', '7']]

In [143]:
print(path_finder(a))

current_node coord = [0, 0]
current_node value = 2
current_node g = 0
current_node h = 40
current_node f = 40


coord = [1, 0]
value = 8
g = 6
h = 39
f = 45
new node g = 6


coord = [0, 1]
value = 2
g = 0
h = 39
f = 39
new node g = 0


current_node coord = [0, 1]
current_node value = 2
current_node g = 0
current_node h = 39
current_node f = 39


coord = [1, 1]
value = 4
g = 2
h = 38
f = 40
new node g = 2


coord = [0, 2]
value = 4
g = 2
h = 38
f = 40
new node g = 2


current_node coord = [0, 2]
current_node value = 4
current_node g = 2
current_node h = 38
current_node f = 40


coord = [1, 2]
value = 8
g = 4
h = 37
f = 41
new node g = 6


coord = [0, 3]
value = 8
g = 4
h = 37
f = 41
new node g = 6


current_node coord = [1, 1]
current_node value = 4
current_node g = 2
current_node h = 38
current_node f = 40


coord = [2, 1]
value = 6
g = 2
h = 37
f = 39
new node g = 4


coord = [1, 2]
value = 8
g = 4
h = 37
f = 41
new node g = 6


coord = [1, 0]
value = 8
g = 4
h = 39
f = 43
new node g 

g = 5
h = 16
f = 21
new node g = 44


current_node coord = [13, 9]
current_node value = 0
current_node g = 38
current_node h = 18
current_node f = 56


coord = [14, 9]
value = 5
g = 5
h = 17
f = 22
new node g = 43


current_node coord = [13, 12]
current_node value = 5
current_node g = 41
current_node h = 15
current_node f = 56


coord = [14, 12]
value = 4
g = 1
h = 14
f = 15
new node g = 42


coord = [12, 12]
value = 6
g = 1
h = 16
f = 17
new node g = 42


*Faster Path* = 42
coord = [13, 13]
value = 3
g = 2
h = 14
f = 16
new node g = 43


current_node coord = [14, 4]
current_node value = 0
current_node g = 34
current_node h = 22
current_node f = 56


coord = [15, 4]
value = 0
g = 0
h = 21
f = 21
new node g = 34


coord = [13, 4]
value = 4
g = 4
h = 23
f = 27
new node g = 38


coord = [14, 5]
value = 3
g = 3
h = 21
f = 24
new node g = 37


current_node coord = [15, 4]
current_node value = 0
current_node g = 34
current_node h = 21
current_node f = 55


coord = [16, 4]
value = 5
g = 5
h =